In [1]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ../TileEM/stored_ptk_run

/home/jlee782/crowd-seg/analysis/TileEM/stored_ptk_run


In [ ]:
ls

In [ ]:
objid=1
niter=3
tiles = pkl.load(open("vtiles{}.pkl".format(objid)))
pInT = pkl.load(open("pInT_lst_obj{0}_iter{1}.pkl".format(objid,niter)))
pNotInT = pkl.load(open("pNotInT_lst_obj{0}_iter{1}.pkl".format(objid,niter)))
Qj = pkl.load(open("Qj_obj{0}_iter{1}.pkl".format(objid,niter)))
#Tstar
Tstar = pkl.load(open("iter_0/tid_list.pkl"))
Tstar2 = pkl.load(open("iter_9/tid_list.pkl"))

### Looking at how different threshold values do for different T* construction thresholds 

In [5]:
objid =
ls 10worker_rand0/obj1/thresh-10/iter_0/

10worker_rand0/  15worker_rand3/  5worker_rand1/  nohup.out
10worker_rand1/  15worker_rand4/  5worker_rand2/  runIndicatorMat.py
10worker_rand2/  15worker_rand5/  5worker_rand3/  sample20.out
10worker_rand3/  20worker_rand0/  5worker_rand4/  sample25_0.out
10worker_rand4/  20worker_rand1/  5worker_rand5/  sampling2.sh
10worker_rand5/  20worker_rand2/  5worker_rand6/  sampling.sh
10worker_rand6/  20worker_rand3/  5worker_rand7/  TileEM_plot_toolbox.py
10worker_rand7/  25worker_rand0/  5worker_rand8/  TileEM_plot_toolbox.pyc
15worker_rand0/  25worker_rand1/  5worker_rand9/
15worker_rand1/  30worker_rand0/  main.py
15worker_rand2/  5worker_rand0/   main.pyc


In [ ]:
precisions = []
recalls = []
for sample_path in glob.glob('{}/*/'.format(indir)):
    print '======================================================'
    sample_name = sample_path.split('/')[-2]
    print 'Processing sample ', sample_name
    # if sample_name != '5worker_rand0':
    #     continue
    # for objid in objects:
    for objid in range(1, 48):
        # if objid != 18:
        #     continue
        print 'Doing object ', objid
        print '-----------------------------------------------------'
        #tiles = pickle.load(open('{}vtiles{}.pkl'.format(sample_path, objid)))
        #indMat = pickle.load(open('{}indMat{}.pkl'.format(sample_path, objid)))
        
        #do stuff
        precisions.append(precision(CORE, GTBB))
        recalls.append(recall(CORE, GTBB))
print 'Precision stats:'
print 'Mean: {}, median: {}, std: {}, min: {}, max: {}'.format(
    np.mean(precisions), np.median(precisions), np.std(precisions), min(precisions), max(precisions))
print 'Recall stats:'
print 'Mean: {}, median: {}, std: {}, min: {}, max: {}'.format(
    np.mean(recalls), np.median(recalls), np.std(recalls), min(recalls), max(recalls))

In [5]:
# Compute PR from all baselines + TileEM
import pickle as pkl 
from TileEM_plot_toolbox import *
from qualityBaseline import *
from glob import glob
from collections import OrderedDict
from bad_vtiles import * 
df = pd.read_csv("../computed_my_COCO_BBvals.csv",index_col=0)
worker_Nbatches={5:10,10:8,15:6,20:4,25:2,30:1}
base_dir= "stored_ptk_run"
sampleN_lst=worker_Nbatches.keys()
#mode="aggregate_sample_table"
mode="recompute_sample_batch_table"

if mode =="recompute_sample_batch_table":
	# Compute and save a  PR_tbl_all.csv into each sample_batch folder 
	for Nworker in sampleN_lst:
		for batch_num in range(worker_Nbatches[Nworker]):
			dir_name = "{0}worker_rand{1}".format(Nworker,batch_num)
			
			print "Working on :", dir_name
			os.chdir(base_dir+"/"+dir_name)
			 
			# Creating 2 empty precision recall table containing P,R for each metric used 
			cols = [u'Num Points',u'Area Ratio',u'Jaccard [Self]', u'Precision [Self]', u'Recall [Self]']

			PR_tbl = pd.DataFrame()#index=object_lst)#,columns=cols)
			PR_tbl["object_id"]=object_lst
			# Summarization-Based Method
			df.ix[df["Precision [Self]"]>1,"Precision [Self]"]=1
			df.ix[df["Recall [Self]"]>1,"Recall [Self]"]=1
			df = df[~(df["worker_id"].isin([1,2,3]))] #Filter our ground truth workers
			for attr in cols:
				tbl=[]
				for objid in object_lst:
					workers=pkl.load(open("worker{}.pkl".format(objid)))
					filtered_df = df[(df["worker_id"].isin(workers))&(df["object_id"]==objid)] #only look at summarization scores of sampled workers
					best_worker_BB = filtered_df[filtered_df[attr]==filtered_df[attr].max()]
					if objid not in [35,41]:
					    tbl.append([objid,best_worker_BB["Precision [Self]"].values[0],best_worker_BB["Recall [Self]"].values[0]])
					else:
					    tbl.append([objid,-1,-1])
				tmp_PR_tbl = pd.DataFrame(tbl,columns=["object_id","Precision","Recall"])
				PR_tbl["P [{}]".format(attr)]=tmp_PR_tbl["Precision"]
				PR_tbl["R [{}]".format(attr)]=tmp_PR_tbl["Recall"]
			# Vision based methods
			print os.getcwd()
			visionPR = pd.read_csv("../../Vision-stuff/VisionGTComparisons/{}/best_Area_Ratio_BBs/box/prec50/PR.csv".format(dir_name))
			PR_tbl["P [Vision Best Area Ratio 50%]"] = visionPR["precision"] 
			PR_tbl["R [Vision Best Area Ratio 50%]"] = visionPR["recall"]
			
			visionPR = pd.read_csv("../../Vision-stuff/VisionGTComparisons/{}/best_Num_Points_BBs/box/prec50/PR.csv".format(dir_name))
			PR_tbl["P [Vision Best Num Points 50%]"] = visionPR["precision"]
                        PR_tbl["R [Vision Best Num Points 50%]"] = visionPR["recall"]	 
            
			for threshold in [10,50,90]:
				visionPR = pd.read_csv("../../../PR{}.csv".format(threshold))
				PR_tbl["P [Vision GT {}%]".format(threshold)] = visionPR["precision"]
				PR_tbl["R [Vision GT {}%]".format(threshold)] = visionPR["recall"]
			#MVT, Tile
			tbl=[]
			for fname in glob("Tstar_idx_obj*.pkl"):
				objid=int(fname.split("_")[-1].split(".")[0][3:])
				tiles = pkl.load(open("vtiles{}.pkl".format(objid)))
				#Tile EM
				Tstar_lst = pkl.load(open("Tstar_idx_obj{}.pkl".format(objid)))
				TileEMP,TileEMR = compute_PR(objid,np.array(Tstar_lst[-1]),tiles)
				# Majority Vote 
				PMVT,RMVT = majority_vote(objid,heuristic="50%")
				PMVTtopk,RMVTtopk = majority_vote(objid,heuristic="topk")
				PMVTtopP,RMVTtopP = majority_vote(objid,heuristic="topPercentile")
				tbl.append([objid,TileEMP,TileEMR,PMVT,RMVT,PMVTtopk,RMVTtopk,PMVTtopP,RMVTtopP])
			Tile_df = pd.DataFrame(tbl,columns=["object_id","P [TileEM]","R [TileEM]","P [MVT]","R [MVT]","P [MVTtop10]","R [MVTtop10]","P [MVTtop95%]","R [MVTtop95%]"])
			PR_tbl_all = PR_tbl.merge(Tile_df,on="object_id")
			#Save to file in that folder 
			PR_tbl_all.to_csv("PR_tbl_all.csv")
			os.chdir("../..")
elif mode =="aggregate_sample_table" :
    sampleN_lst=sorted(worker_Nbatches.keys())

    for Nworker in sampleN_lst:
        print "Working on worker = ",Nworker
        batch_all_data=[]
        for batch_num in range(worker_Nbatches[Nworker]):
            dir_name = "sample/{0}worker_rand{1}".format(Nworker,batch_num)
            batch_i_data=pd.read_csv(dir_name+"/PR_tbl_all.csv",index_col=0)
            #Drop the object rows where the objects have bad vtiles to begin with 
            bad_vtile_objs = list(problematic[(problematic["Nworker"]==Nworker)&(problematic["batch_num"]==batch_num)].objid)
	    bad_vtile_objs.extend([35,41]) # Discard ambiguous objects 
            batch_i_data=batch_i_data.drop(bad_vtile_objs,errors='ignore')
            #batch_i_data[~batch_i_data["object_id"].isin(bad_vtile_objs)]
            # First set the object_id column the index, then reindex based on this index to correspond to object lst
            # This fills in a row of NaN for the missing data objects, so it normalizes the shape of our data table 
            batch_i_data = batch_i_data.set_index('object_id').reindex(object_lst,fill_value =0)
             # Add a row of non-nan count for averaging 
            nan_rowidx= list(batch_i_data[batch_i_data["P [Num Points]"]==0].index)
            non_nan_count = np.ones_like(object_lst)
            #non_nan_count = np.ones(47)
	    for row in nan_rowidx: non_nan_count[row-1]=0
            batch_i_data["non_nan_count"]=non_nan_count
            if batch_num==0:
                batch_all_data=batch_i_data
            else:
                batch_all_data+=batch_i_data
        # Renaming data columns
        df = df.rename(index=str,columns={'P [Jaccard [Self]]':'P [GT Jaccard]','R [Jaccard [Self]]':'R [GT Jaccard]',\
                             'P [Precision [Self]]':'P [GT Precision]','R [Precision [Self]]':'R [GT Precision]',\
                             'P [Recall [Self]]':'P [GT Recall]','R [Recall [Self]]':'R [GT Recall]',\
                            })
        try:
            batch_all_data=batch_all_data[batch_all_data.keys()[:-1]].divide(batch_all_data["non_nan_count"],axis=0)
            batch_all_data[batch_all_data>1]=1
            batch_all_data[batch_all_data<0]=0
	    print "Created sample{}_PR.csv".format(Nworker)
            batch_all_data.to_csv("sample{}_PR.csv".format(Nworker))
        except(ZeroDivisionError):
            print "No data for worker=",Nworker 




Working on : 5worker_rand0


TopologyException: side location conflict at 447 175
TopologyException: side location conflict at 461 219
TopologyException: found non-noded intersection between LINESTRING (398.6 163.8, 398.087 159.696) and LINESTRING (398.087 159.696, 398.812 171.055) at 398.08695652173913 159.69565217391317
TopologyException: found non-noded intersection between LINESTRING (373 126, 372.353 127.294) and LINESTRING (372.353 127.294, 376.667 126) at 372.35294117647061 127.29411764705883
Self-intersection at or near point 372.35294117647061 127.29411764705884
TopologyException: found non-noded intersection between LINESTRING (398.6 163.8, 398.087 159.696) and LINESTRING (398.087 159.696, 398.812 171.055) at 398.08695652173913 159.69565217391317


/home/jlee782/crowd-seg/analysis/TileEM/sample/5worker_rand0


TopologyException: found non-noded intersection between LINESTRING (169.863 171.003, 170.721 171.465) and LINESTRING (175 172, 170.721 171.465) at 170.72093023255812 171.46511627906975
TopologyException: found non-noded intersection between LINESTRING (169.863 171.003, 170.721 171.465) and LINESTRING (175 172, 170.721 171.465) at 170.72093023255812 171.46511627906975
TopologyException: found non-noded intersection between LINESTRING (170.721 171.465, 170.721 171.465) and LINESTRING (170.721 171.465, 170.721 171.465) at 170.72093023255815 171.46511627906978
TopologyException: side location conflict at 334.22222222222217 146.55555555555554
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (314.5 150.5, 310 155) at 310 154.99999999999994


TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310

Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (393.713 240.796, 413.795 264.783) and LINESTRING (387.818 233.755, 416.019 267.439) at 398.69066629504437 246.74162918574746
TopologyException: found non-noded intersection between LINESTRING (367.388 211.699, 368 217) and LINESTRING (367.388 211.699, 368.739 214.13) at 367.38834951456312 211.69902912621365
Self-intersection at or near point 367.38834951456312 211.69902912621365
TopologyException: found non-noded intersection between LINESTRING (367.388 211.699,

Self-intersection at or near point 367.38834951456312 211.69902912621365
TopologyException: found non-noded intersection between LINESTRING (367.388 211.699, 368 217) and LINESTRING (367.388 211.699, 368.739 214.13) at 367.38834951456312 211.69902912621365
Self-intersection at or near point 367.38834951456312 211.69902912621365
TopologyException: found non-noded intersection between LINESTRING (367.388 211.699, 368 217) and LINESTRING (367.388 211.699, 368.739 214.13) at 367.38834951456312 211.69902912621365
Self-intersection at or near point 367.38834951456312 211.69902912621365
TopologyException: found non-noded intersection between LINESTRING (367.388 211.699, 368 217) and LINESTRING (367.388 211.699, 368.739 214.13) at 367.38834951456312 211.69902912621365
Self-intersection at or near point 367.38834951456312 211.69902912621365
TopologyException: found non-noded intersection between LINESTRING (367.388 211.699, 368 217) and LINESTRING (367.388 211.699, 368.739 214.13) at 367.388349

KeyboardInterrupt: 

In [16]:
cd ../../

/home/jlee782/crowd-seg/analysis/TileEM


In [15]:
base_dir= "stored_ptk_run"
# Compute and save a  PR_tbl_all.csv into each sample_batch folder 
for Nworker in [10]:#sampleN_lst:
    for batch_num in range(worker_Nbatches[Nworker]):
        dir_name = "{0}worker_rand{1}".format(Nworker,batch_num)

        print "Working on :", dir_name
        os.chdir(base_dir+"/"+dir_name)

        # Creating 2 empty precision recall table containing P,R for each metric used 
        cols = [u'Num Points',u'Area Ratio',u'Jaccard [Self]', u'Precision [Self]', u'Recall [Self]']

        PR_tbl = pd.DataFrame()#index=object_lst)#,columns=cols)
        PR_tbl["object_id"]=object_lst
        # Summarization-Based Method
        df.ix[df["Precision [Self]"]>1,"Precision [Self]"]=1
        df.ix[df["Recall [Self]"]>1,"Recall [Self]"]=1
        df = df[~(df["worker_id"].isin([1,2,3]))] #Filter our ground truth workers
        for attr in cols:
            tbl=[]
            for objid in object_lst:
                workers=pkl.load(open("worker{}.pkl".format(objid)))
                filtered_df = df[(df["worker_id"].isin(workers))&(df["object_id"]==objid)] #only look at summarization scores of sampled workers
                best_worker_BB = filtered_df[filtered_df[attr]==filtered_df[attr].max()]
                if objid not in [35,41]:
                    tbl.append([objid,best_worker_BB["Precision [Self]"].values[0],best_worker_BB["Recall [Self]"].values[0]])
                else:
                    tbl.append([objid,-1,-1])
            tmp_PR_tbl = pd.DataFrame(tbl,columns=["object_id","Precision","Recall"])
            PR_tbl["P [{}]".format(attr)]=tmp_PR_tbl["Precision"]
            PR_tbl["R [{}]".format(attr)]=tmp_PR_tbl["Recall"]
        # Vision based methods
        print os.getcwd()
        visionPR = pd.read_csv("../../Vision-stuff/VisionGTComparisons/{}/best_Area_Ratio_BBs/box/prec50/PR.csv".format(dir_name))
        PR_tbl["P [Vision Best Area Ratio 50%]"] = visionPR["precision"] 
        PR_tbl["R [Vision Best Area Ratio 50%]"] = visionPR["recall"]

        visionPR = pd.read_csv("../../Vision-stuff/VisionGTComparisons/{}/best_Num_Points_BBs/box/prec50/PR.csv".format(dir_name))
        PR_tbl["P [Vision Best Num Points 50%]"] = visionPR["precision"]
        PR_tbl["R [Vision Best Num Points 50%]"] = visionPR["recall"]

        for threshold in [10,50,90]:
            visionPR = pd.read_csv("../../../PR{}.csv".format(threshold))
            PR_tbl["P [Vision GT {}%]".format(threshold)] = visionPR["precision"]
            PR_tbl["R [Vision GT {}%]".format(threshold)] = visionPR["recall"]
        #MVT, Tile
        tbl=[]
        for fname in glob("Tstar_idx_obj*.pkl"):
            objid=int(fname.split("_")[-1].split(".")[0][3:])
            tiles = pkl.load(open("vtiles{}.pkl".format(objid)))
            #Tile EM
            Tstar_lst = pkl.load(open("Tstar_idx_obj{}.pkl".format(objid)))
            TileEMP,TileEMR = compute_PR(objid,np.array(Tstar_lst[-1]),tiles)
            # Majority Vote 
            PMVT,RMVT = majority_vote(objid,heuristic="50%")
            PMVTtopk,RMVTtopk = majority_vote(objid,heuristic="topk")
            PMVTtopP,RMVTtopP = majority_vote(objid,heuristic="topPercentile")
            tbl.append([objid,TileEMP,TileEMR,PMVT,RMVT,PMVTtopk,RMVTtopk,PMVTtopP,RMVTtopP])
        Tile_df = pd.DataFrame(tbl,columns=["object_id","P [TileEM]","R [TileEM]","P [MVT]","R [MVT]","P [MVTtop10]","R [MVTtop10]","P [MVTtop95%]","R [MVTtop95%]"])
        PR_tbl_all = PR_tbl.merge(Tile_df,on="object_id")
        #Save to file in that folder 
        PR_tbl_all.to_csv("PR_tbl_all.csv")
        os.chdir("../..")

Working on : 10worker_rand0


OSError: [Errno 2] No such file or directory: 'stored_ptk_run/10worker_rand0'